In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
survey = pd.read_csv("survey_results_public.csv")
survey_ind = survey[~(survey['Industry'].isna())].copy()
survey_comp = survey_ind[~(survey_ind['ConvertedCompYearly'].isna())]
degree_map = {
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": "Bachelor's",
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": "Master's",
    "Some college/university study without earning a degree": "Some College",
    "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)": "Secondary School",
    "Professional degree (JD, MD, Ph.D, Ed.D, etc.)": "Professional Degree",
    "Associate degree (A.A., A.S., etc.)": "Associate Degree",
    "Something else": "Other",
    "Primary/elementary school": "Primary School"
}

survey_ind["EdLevel"] = survey_ind["EdLevel"].replace(degree_map)
survey_ed = survey_ind[survey_ind["EdLevel"].isin(["Bachelor's", "Master's", "Some College", "Secondary School", "Professional Degree",
                                                   "Associate Degree"])]
data = survey_ed[["Industry", "EdLevel", "YearsCode", "ConvertedCompYearly", "LanguageHaveWorkedWith", "NEWCollabToolsHaveWorkedWith", 
                  "AISearchDevHaveWorkedWith"]]
data = data.rename({"ConvertedCompYearly":"Salary", "LanguageHaveWorkedWith":"Languages", "NEWCollabToolsHaveWorkedWith":"CodeEditors",
             "AISearchDevHaveWorkedWith":"AiTools"}, axis=1)
data["YearsCode"] = pd.to_numeric(data["YearsCode"], errors="coerce", downcast='integer')
data["Salary"] = pd.to_numeric(data["Salary"], errors="coerce", downcast='integer')
data["Industry"] = data["Industry"].replace({'Internet, Telecomm or Information Services':'Internet/Telecomm/Info Services', 
                                             'Retail and Consumer Services':'Retail & Consumer Services', 
                                             'Computer Systems Design and Services':'Computer Systems Design & Services',
                                             'Transportation, or Supply Chain':'Transportation/Supply Chain'})
data = data[~(data["Industry"] == 'Other:')]
data = data.reset_index()
data.head()

,index,Industry,EdLevel,YearsCode,Salary,Languages,CodeEditors,AiTools
0,10,Healthcare,Bachelor's,20.0,NaN,JavaScript;Python;Ruby,Visual Studio Code,ChatGPT;GitHub Copilot;Google Gemini;Meta AI;V...
1,12,Software Development,Bachelor's,20.0,NaN,Bash/Shell (all shells);C#;JavaScript;PowerShe...,Notepad++;Vim;Visual Studio Code,NaN
2,15,Banking/Financial Services,Some College,31.0,NaN,C#;HTML/CSS;JavaScript;PowerShell;SQL;TypeScri...,Notepad++;Visual Studio;Visual Studio Code,ChatGPT;Google Gemini;Visual Studio Intellicode
3,18,Software Development,Some College,12.0,NaN,C#;HTML/CSS;JavaScript;PHP;Python;SQL,Visual Studio;Visual Studio Code,ChatGPT;GitHub Copilot
4,22,Banking/Financial Services,Bachelor's,25.0,NaN,Bash/Shell (all shells);C#;F#;JavaScript;Power...,Rider;Visual Studio Code,GitHub Copilot


In [3]:
def ohe(column_name, prefix):
    temp = data[[column_name]].copy()
    temp[column_name] = temp[column_name].str.split(";")
    exp = temp.explode(column_name)
    dummies = pd.get_dummies(exp[column_name], prefix=prefix)
    mhe = dummies.groupby(exp.index).sum()
    return mhe

lang = ohe("Languages", "Lang")
editor = ohe("CodeEditors", "Editor")
Ai = ohe("AiTools", "AI")

data2 = data.join([lang, editor, Ai])
data2.head()

,index,Industry,EdLevel,YearsCode,Salary,Languages,CodeEditors,AiTools,Lang_Ada,Lang_Apex,...,AI_Perplexity AI,AI_Phind,AI_Quora Poe,AI_Replit Ghostwriter,AI_Snyk Code,AI_Tabnine,AI_Visual Studio Intellicode,AI_Whispr AI,AI_WolframAlpha,AI_You.com
0,10,Healthcare,Bachelor's,20.0,NaN,JavaScript;Python;Ruby,Visual Studio Code,ChatGPT;GitHub Copilot;Google Gemini;Meta AI;V...,0,0,...,0,0,0,0,0,0,1,0,0,0
1,12,Software Development,Bachelor's,20.0,NaN,Bash/Shell (all shells);C#;JavaScript;PowerShe...,Notepad++;Vim;Visual Studio Code,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,Banking/Financial Services,Some College,31.0,NaN,C#;HTML/CSS;JavaScript;PowerShell;SQL;TypeScri...,Notepad++;Visual Studio;Visual Studio Code,ChatGPT;Google Gemini;Visual Studio Intellicode,0,0,...,0,0,0,0,0,0,1,0,0,0
3,18,Software Development,Some College,12.0,NaN,C#;HTML/CSS;JavaScript;PHP;Python;SQL,Visual Studio;Visual Studio Code,ChatGPT;GitHub Copilot,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22,Banking/Financial Services,Bachelor's,25.0,NaN,Bash/Shell (all shells);C#;F#;JavaScript;Power...,Rider;Visual Studio Code,GitHub Copilot,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
results = []

for industry in data2['Industry'].unique():
    for ed in data2['EdLevel'].unique():
        df = data2[(data2['Industry'] == industry) & (data2['EdLevel'] == ed)]
        low, high = df['Salary'].quantile([0.01, 0.99])
        mask = df['Salary'].between(low, high) | df['Salary'].isna()
        results.append(df[mask])

percentiles = pd.concat(results, ignore_index=True)

In [7]:
percentiles.to_csv("stack_overflow_survey2.csv", index=False)

In [2]:
# Printing name lists to manually put in Sketchingpy Code
# industries = data['Industry'].unique().tolist()
# lang_cols = [col for col in data2.columns if col.startswith('Lang_')]
# ai_cols = [col for col in data2.columns if col.startswith('AI_')]
# editor_cols = [col for col in data2.columns if col.startswith('Editor_')]
# education = data['EdLevel'].unique().tolist()
# education